In [3]:
import pandas as pd
from datetime import date, datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, input_file_name, split, explode, explode_outer, col

spark = SparkSession.builder.getOrCreate()
# date_recent = datetime.strptime("2023-03-28", '%Y-%m-%d').date()
# folder_path = "abfss://raw@secjurdatalakestage.dfs.core.windows.net/secjur-deu/DCO/tasks/"

your 131072x1 screen size is bogus. expect trouble
23/04/28 19:00:08 WARN Utils: Your hostname, LAPTOP-8FLA7LTL resolves to a loopback address: 127.0.1.1; using 172.22.124.54 instead (on interface eth0)
23/04/28 19:00:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 19:00:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


def flatten_explode(df):
    while True:
        complex_field = next(((field.name, field.dataType)
                              for field in df.schema.fields
                              if isinstance(field.dataType, (ArrayType, StructType))), None)
        if not complex_field:
            break

        col_name, col_type = complex_field

        if isinstance(col_type, StructType):
            expanded = [col(f"{col_name}.{k}").alias(f"{col_name}_{k}")
                        for k in [n.name for n in col_type]]
            df = df.select("*", *expanded).drop(col_name)
        elif isinstance(col_type, ArrayType):
            df = df.withColumn(col_name, explode_outer(col_name))

    return df


def custom_rename(old_name, old_prefix, prefix):
    if old_name not in ["subtasks_id"]:
        return old_name.replace(old_prefix, prefix)
    else:
        return f""" {prefix}_{old_name} """
    
    
def add_naming_convention(df):
    
    for column_name in df.columns:
        df = df.withColumnRenamed(
            column_name, custom_rename(column_name, "subtasks", "task")
        )


    df = df.withColumnRenamed(
        "assignee_id", 'task_assignee_id'
    ).withColumnRenamed("assignee_type", 'task_assignee_type')

    try:
        df = (
            df
            .withColumn("comment_id", col("comment.id"))
            .withColumn("comment_text", col("comment.text"))
            .withColumn("comment_created_at_date", col("comment.created_at_date"))
            .withColumn("comment_created_at_time", col("comment.created_at_time"))
            .withColumn("comment_created_by", col("comment.created_by"))
            .drop("comment")
        )
    except:
        pass


    return df


nested_json_file_path = "task_example.json"
df = spark.read.json(path=nested_json_file_path, multiLine=True)
df = df.select("results")
df = df.selectExpr("explode(results) as results")
df = df.selectExpr(
    "results.assignees as assignees", 
    "results.subtasks as subtasks", 
    "results.comments as comment", 
    "results.tag as tag", 
    "results.id as id", 
    "results.task_id as task_id")

df = flatten_explode(df)
df = add_naming_convention(df)

desc_stats = df.describe()
desc_stats_pd = desc_stats.toPandas()
desc_stats_pd_transposed = desc_stats_pd.transpose()
desc_stats_pd_transposed.to_csv("output.csv")